#Trying Jupyter Lab

In [1]:
%reload_kedro

2021-10-08 15:15:00,347 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2021-10-08 15:15:00,397 - root - INFO - ** Kedro project spaceflightsTutorial
2021-10-08 15:15:00,399 - root - INFO - Defined global variable `context`, `session` and `catalog`
2021-10-08 15:15:00,408 - root - INFO - Registered line magic `run_viz`


In [2]:
session.run()

2021-10-08 15:15:00,415 - root - INFO - ** Kedro project spaceflightsRepo
2021-10-08 15:15:00,817 - kedro.io.data_catalog - INFO - Loading data from `shuttles` (ExcelDataSet)...


/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflights-venv/lib/python3.8/site-packages/kedro/extras/datasets/pandas/excel_dataset.py:168: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. Install openpyxl instead.
  return pd.read_excel(fs_file, **self._load_args)
/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflights-venv/lib/python3.8/site-packages/xlrd/xlsx.py:39: DeprecationWarning: defusedxml.cElementTree is deprecated, import from defusedxml.ElementTree instead.
  try: import defusedxml.cElementTree as ET
/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflights-venv/lib/python3.8/site-packages/xlrd/xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflights-venv/lib/python3.8/site-packages/xlrd/xlsx.py:312: 

2021-10-08 15:15:09,731 - kedro.pipeline.node - INFO - Running node: preprocess_shuttles_node: preprocess_shuttles([shuttles]) -> [preprocessed_shuttles]
2021-10-08 15:15:09,807 - kedro.io.data_catalog - INFO - Saving data to `preprocessed_shuttles` (CSVDataSet)...


/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflightsRepo/src/process/pipelines/data_processing/nodes.py:42: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace("$", "").str.replace(",", "")


2021-10-08 15:15:10,155 - kedro.runner.sequential_runner - INFO - Completed 1 out of 8 tasks
2021-10-08 15:15:10,156 - kedro.io.data_catalog - INFO - Loading data from `companies` (CSVDataSet)...
2021-10-08 15:15:10,187 - kedro.pipeline.node - INFO - Running node: preprocess_companies_node: preprocess_companies([companies]) -> [preprocessed_companies]
2021-10-08 15:15:10,220 - kedro.io.data_catalog - INFO - Saving data to `preprocessed_companies` (CSVDataSet)...
2021-10-08 15:15:10,407 - kedro.runner.sequential_runner - INFO - Completed 2 out of 8 tasks
2021-10-08 15:15:10,408 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_shuttles` (CSVDataSet)...
2021-10-08 15:15:10,505 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_companies` (CSVDataSet)...
2021-10-08 15:15:10,539 - kedro.io.data_catalog - INFO - Loading data from `reviews` (CSVDataSet)...
2021-10-08 15:15:10,601 - kedro.pipeline.node - INFO - Running node: create_model_input_table_node: create_m

{}

In [3]:
context.catalog

In [4]:
context.pipeline

/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflights-venv/lib/python3.8/site-packages/kedro/framework/context/context.py:59: DeprecationWarning: Accessing pipeline via the context will be deprecated in Kedro 0.18.0.
  warn(


Pipeline([
Node(preprocess_shuttles, 'shuttles', 'preprocessed_shuttles', 'preprocess_shuttles_node'),
Node(preprocess_companies, 'companies', 'preprocessed_companies', 'preprocess_companies_node'),
Node(create_model_input_table, ['preprocessed_shuttles', 'preprocessed_companies', 'reviews'], 'model_input_table', 'create_model_input_table_node'),
Node(split_data, ['model_input_table', 'parameters'], ['X_train', 'X_test', 'y_train', 'y_test'], 'split_data_node'),
Node(train_model, ['X_train', 'y_train'], 'regressor', 'train_model_node'),
Node(train_model_RF, ['X_train', 'y_train'], 'RF_regressor', 'train_model_RF_node'),
Node(evaluate_model, ['regressor', 'X_test', 'y_test'], None, 'evaluate_model_node'),
Node(evaluate_model_RF, ['RF_regressor', 'X_test', 'y_test'], None, 'evaluate_model_RF_node')
])

In [5]:
context.project_path

PosixPath('/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflightsRepo')

In [6]:
context.params

{'test_size': 0.2,
 'random_state': 3,
 'features': ['engines',
  'passenger_capacity',
  'crew',
  'd_check_complete',
  'moon_clearance_complete',
  'iata_approved',
  'company_rating',
  'review_scores_rating']}

In [7]:
companies_df = catalog.load("companies")
reviews_df = catalog.load("reviews")
shuttles_df = catalog.load("shuttles")

2021-10-08 15:16:06,547 - kedro.io.data_catalog - INFO - Loading data from `companies` (CSVDataSet)...
2021-10-08 15:16:06,578 - kedro.io.data_catalog - INFO - Loading data from `reviews` (CSVDataSet)...
2021-10-08 15:16:06,642 - kedro.io.data_catalog - INFO - Loading data from `shuttles` (ExcelDataSet)...


/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflights-venv/lib/python3.8/site-packages/kedro/extras/datasets/pandas/excel_dataset.py:168: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. Install openpyxl instead.
  return pd.read_excel(fs_file, **self._load_args)
/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflights-venv/lib/python3.8/site-packages/xlrd/xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
/Users/adityan/Work/kedro-spaceflights-tutorial/spaceflights-venv/lib/python3.8/site-packages/xlrd/xlsx.py:312: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


In [8]:
companies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77096 entries, 0 to 77095
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 77096 non-null  int64  
 1   company_rating     47187 non-null  object 
 2   company_location   57966 non-null  object 
 3   total_fleet_count  77089 non-null  float64
 4   iata_approved      77089 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.9+ MB


In [9]:
companies_df.isnull().sum()

id                       0
company_rating       29909
company_location     19130
total_fleet_count        7
iata_approved            7
dtype: int64

In [10]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77096 entries, 0 to 77095
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   shuttle_id               77096 non-null  int64  
 1   review_scores_rating     55956 non-null  float64
 2   review_scores_comfort    55896 non-null  float64
 3   review_scores_amenities  55909 non-null  float64
 4   review_scores_trip       55833 non-null  float64
 5   review_scores_crew       55902 non-null  float64
 6   review_scores_location   55831 non-null  float64
 7   review_scores_price      55828 non-null  float64
 8   number_of_reviews        77096 non-null  int64  
 9   reviews_per_month        57553 non-null  float64
dtypes: float64(8), int64(2)
memory usage: 5.9 MB


In [11]:
reviews_df.isnull().sum()

shuttle_id                     0
review_scores_rating       21140
review_scores_comfort      21200
review_scores_amenities    21187
review_scores_trip         21263
review_scores_crew         21194
review_scores_location     21265
review_scores_price        21268
number_of_reviews              0
reviews_per_month          19543
dtype: int64

In [12]:
shuttles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77096 entries, 0 to 77095
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       77096 non-null  int64  
 1   shuttle_location         77096 non-null  object 
 2   shuttle_type             77096 non-null  object 
 3   engine_type              77096 non-null  object 
 4   engine_vendor            77096 non-null  object 
 5   engines                  77057 non-null  float64
 6   passenger_capacity       77096 non-null  int64  
 7   cancellation_policy      77096 non-null  object 
 8   crew                     76947 non-null  float64
 9   d_check_complete         77096 non-null  object 
 10  moon_clearance_complete  77096 non-null  object 
 11  price                    77096 non-null  object 
 12  company_id               77096 non-null  int64  
dtypes: float64(2), int64(3), object(8)
memory usage: 7.6+ MB


In [13]:
shuttles_df.isnull().sum()

id                           0
shuttle_location             0
shuttle_type                 0
engine_type                  0
engine_vendor                0
engines                     39
passenger_capacity           0
cancellation_policy          0
crew                       149
d_check_complete             0
moon_clearance_complete      0
price                        0
company_id                   0
dtype: int64

In [14]:
companies_df.head()

,id,company_rating,company_location,total_fleet_count,iata_approved
0,35029,100%,Niue,4.0,f
1,30292,67%,Anguilla,6.0,f
2,19032,67%,Russian Federation,4.0,f
3,8238,91%,Barbados,15.0,t
4,30342,NaN,Sao Tome and Principe,2.0,t


In [15]:
params_dict = context.params 
print(params_dict)
for k,v in params_dict.items():
    print(k,v)

{'test_size': 0.2, 'random_state': 3, 'features': ['engines', 'passenger_capacity', 'crew', 'd_check_complete', 'moon_clearance_complete', 'iata_approved', 'company_rating', 'review_scores_rating']}
test_size 0.2
random_state 3
features ['engines', 'passenger_capacity', 'crew', 'd_check_complete', 'moon_clearance_complete', 'iata_approved', 'company_rating', 'review_scores_rating']


In [16]:
# I tried running the pipeline 'dp' to capture the output from the final pipeline, but it session.run() doesnt return output
#model_input_table = session.run(pipeline_name='dp')
# I tried running the specific nodes to capture the output, but again no output captured
#model_input_table = session.run(node_names=['preprocess_companies_node','preprocess_shuttles_node','create_model_input_table_node'])
#model_input_table = context.run(node_names=['preprocess_companies_node'])

#To access the intermediate dataset, we can directly use catalog.load("<filename>") -- just like we did for raw inputfiles. 
#We dont need to explicitly run the pipeline or nodes to generate output 

model_input_table = catalog.load("model_input_table")

2021-10-08 15:16:15,013 - kedro.io.data_catalog - INFO - Loading data from `model_input_table` (CSVDataSet)...


In [17]:
model_input_table

,id_x,shuttle_location,shuttle_type,engine_type,engine_vendor,engines,passenger_capacity,cancellation_policy,crew,d_check_complete,...,review_scores_crew,review_scores_location,review_scores_price,number_of_reviews,reviews_per_month,id_y,company_rating,company_location,total_fleet_count,iata_approved
0,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
1,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
2,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
3,63561,Niue,Type V5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,9.0,10.0,133,1.65,35029,1.0,Niue,4.0,False
4,53260,Niue,Type V5,Quantum,"Banks, Wood and Phillips",1.0,2,strict,1.0,False,...,10.0,9.0,10.0,37,0.48,35029,1.0,Niue,4.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759604,39094,Russian Federation,Type F5,Quantum,ThetaBase Services,1.0,2,strict,1.0,False,...,10.0,10.0,10.0,1,1.00,42904,0.7,Russian Federation,2.0,True
759605,20330,Uzbekistan,Type V5,Quantum,ThetaBase Services,1.0,2,flexible,1.0,False,...,10.0,10.0,10.0,1,1.00,5701,1.0,Costa Rica,1.0,True
759606,16445,Nicaragua,Type V5,Plasma,ThetaBase Services,1.0,1,flexible,3.0,False,...,10.0,10.0,10.0,3,3.00,13728,1.0,Pakistan,1.0,False
759607,76469,Bouvet Island (Bouvetoya),Type V5,Quantum,ThetaBase Services,1.0,2,moderate,1.0,False,...,10.0,10.0,10.0,1,1.00,41714,1.0,Lebanon,1.0,False


In [18]:
model_input_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759609 entries, 0 to 759608
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id_x                     759609 non-null  int64  
 1   shuttle_location         759609 non-null  object 
 2   shuttle_type             759609 non-null  object 
 3   engine_type              759609 non-null  object 
 4   engine_vendor            759609 non-null  object 
 5   engines                  759609 non-null  float64
 6   passenger_capacity       759609 non-null  int64  
 7   cancellation_policy      759609 non-null  object 
 8   crew                     759609 non-null  float64
 9   d_check_complete         759609 non-null  bool   
 10  moon_clearance_complete  759609 non-null  bool   
 11  price                    759609 non-null  float64
 12  company_id               759609 non-null  int64  
 13  shuttle_id               759609 non-null  int64  
 14  revi

In [19]:
y = model_input_table.price
X = model_input_table.drop(['price','id_x','company_id','shuttle_id','id_y'],axis=1)

In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759609 entries, 0 to 759608
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   shuttle_location         759609 non-null  object 
 1   shuttle_type             759609 non-null  object 
 2   engine_type              759609 non-null  object 
 3   engine_vendor            759609 non-null  object 
 4   engines                  759609 non-null  float64
 5   passenger_capacity       759609 non-null  int64  
 6   cancellation_policy      759609 non-null  object 
 7   crew                     759609 non-null  float64
 8   d_check_complete         759609 non-null  bool   
 9   moon_clearance_complete  759609 non-null  bool   
 10  review_scores_rating     759609 non-null  float64
 11  review_scores_comfort    759609 non-null  float64
 12  review_scores_amenities  759609 non-null  float64
 13  review_scores_trip       759609 non-null  float64
 14  revi

In [21]:
#Convert Bool values to 0-1
def bool_to_int(df):
    bool_col = [col for col in df.columns if df[col].dtypes == 'bool']
    print(bool_col)
    for col in bool_col:
        df[col] = df[col].astype('int64')
    return df

X = bool_to_int(X)

['d_check_complete', 'moon_clearance_complete', 'iata_approved']


In [22]:
cat_cols = [col for col in X.columns if X[col].dtypes == 'object' and X[col].nunique() <= 10]
num_cols = [col for col in X.columns if X[col].dtypes in ('int64','float64')]
print("Categorical columns :",cat_cols)
print("Numerical columns   :",num_cols)

Categorical columns : ['engine_type', 'engine_vendor', 'cancellation_policy']
Numerical columns   : ['engines', 'passenger_capacity', 'crew', 'd_check_complete', 'moon_clearance_complete', 'review_scores_rating', 'review_scores_comfort', 'review_scores_amenities', 'review_scores_trip', 'review_scores_crew', 'review_scores_location', 'review_scores_price', 'number_of_reviews', 'reviews_per_month', 'company_rating', 'total_fleet_count', 'iata_approved']


In [23]:
X.shape

(759609, 23)

In [24]:
X['shuttle_location'].value_counts()

Barbados                     120776
Micronesia                   109398
Malta                        106708
Nicaragua                     92759
Rwanda                        69383
Russian Federation            58490
Sao Tome and Principe         50293
United Kingdom                28488
Niue                          26269
Bouvet Island (Bouvetoya)     23346
Wallis and Futuna             14835
Indonesia                     10169
Kiribati                       9473
Anguilla                       7437
Uzbekistan                     6505
Gambia                         5454
Guinea                         4346
Costa Rica                     4057
Faroe Islands                  3973
Estonia                        2380
Isle of Man                    1142
Switzerland                     892
Bosnia and Herzegovina          792
Marshall Islands                525
Brazil                          514
Kenya                           458
Pakistan                        221
El Salvador                 

In [25]:
X['shuttle_type'].value_counts()

Type V5    504840
Type F5    187107
Type G0     48629
Type V2      8887
Type Z6      2779
Type O3      1876
Type V7      1500
Type N0      1323
Type X3       547
Type E3       474
Type F1       348
Type W5       284
Type S7       242
Type A7       238
Type U1       109
Type L7       108
Type Y7        67
Type Z1        53
Type Y4        31
Type B7        30
Type I5        28
Type T7        27
Type D4        21
Type P6        19
Type I7        13
Type L0        11
Type O0         8
Type D2         4
Type W4         3
Type K2         1
Type F6         1
Type C2         1
Name: shuttle_type, dtype: int64

In [26]:
X['engine_type'].value_counts()

Plasma     669699
Quantum     83227
Nuclear      6683
Name: engine_type, dtype: int64

In [27]:
X['engine_vendor'].value_counts()

ThetaBase Services                  758528
Banks, Wood and Phillips               797
Warwick Technology Multinational       168
SIT Technology Unlimited                81
MCW Global                              35
Name: engine_vendor, dtype: int64

In [28]:
X['cancellation_policy'].value_counts()

strict      673660
moderate     55883
flexible     30066
Name: cancellation_policy, dtype: int64

In [29]:
X = X[cat_cols + num_cols].copy()

In [30]:
X.shape

(759609, 20)

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print("X_train shape:",X_train.shape)
print("y_train shape:",y_train.shape)
print("X_test shape:",X_test.shape)
print("y_test shape:",X_test.shape)

X_train shape: (531726, 20)
y_train shape: (531726,)
X_test shape: (227883, 20)
y_test shape: (227883, 20)


In [33]:
#Preprocessing Numerical Data
num_transformer = Pipeline(steps=[
                                  ('scaler',StandardScaler())
                                  ])

#Preprocessing Categorical Data
cat_transformer = Pipeline(steps=[
                                  ('onehot',OneHotEncoder(handle_unknown='ignore'))
                                  ])		
		


#Bundle Preprocessing steps
preprocessor = ColumnTransformer(transformers=[
                                               ('num',num_transformer,num_cols),
                                               ('cat',cat_transformer,cat_cols)
])


In [34]:
RFregressor = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                       max_depth=300, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

pipeline_RFregressor = Pipeline(steps=[('preprocessor', preprocessor),
                           ('RFregressor', RFregressor)])

In [35]:
pipeline_RFregressor.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['engines',
                                                   'passenger_capacity', 'crew',
                                                   'd_check_complete',
                                                   'moon_clearance_complete',
                                                   'review_scores_rating',
                                                   'review_scores_comfort',
                                                   'review_scores_amenities',
                                                   'review_scores_trip',
                                                   'review_scores_crew',
                                                   'review_scores_location',
 

In [36]:
y_pred = pipeline_RFregressor.predict(X_test)
score = r2_score(y_test, y_pred)

In [37]:
print("RFregressorModel with more features have a coefficient R^2 of %.3f on test data.", score)

RFregressorModel with more features have a coefficient R^2 of %.3f on test data. 0.9891414953454817


In [38]:
importance = RFregressor.feature_importances_

In [39]:
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.39825
Feature: 1, Score: 0.06526
Feature: 2, Score: 0.02813
Feature: 3, Score: 0.00843
Feature: 4, Score: 0.00000
Feature: 5, Score: 0.04550
Feature: 6, Score: 0.01702
Feature: 7, Score: 0.01274
Feature: 8, Score: 0.01586
Feature: 9, Score: 0.01017
Feature: 10, Score: 0.02629
Feature: 11, Score: 0.02352
Feature: 12, Score: 0.03284
Feature: 13, Score: 0.11107
Feature: 14, Score: 0.00866
Feature: 15, Score: 0.11413
Feature: 16, Score: 0.00526
Feature: 17, Score: 0.00004
Feature: 18, Score: 0.07243
Feature: 19, Score: 0.00016
Feature: 20, Score: 0.00000
Feature: 21, Score: 0.00000
Feature: 22, Score: 0.00000
Feature: 23, Score: 0.00000
Feature: 24, Score: 0.00000
Feature: 25, Score: 0.00101
Feature: 26, Score: 0.00175
Feature: 27, Score: 0.00146
